# Connect to running JVM

In [1]:
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path

In [2]:
from docria.printout import options

Printout settings are located in options as included above

```py
options.max_rows = None
options.max_column_width = None
options.max_display_width = None
```

Disables all output truncation, but if you have 10000+ nodes in a layer, your browser might not be happy with displaying it.

```py
    set_large_screen() 
```

Provides a better default if you have the screen space for it.

In [3]:
set_large_screen()

Py4j connects to a running JVM.

It is the same as starting your program with Java, and can be started from your IDE of choice, then allows debugging and breakpoints when triggered by python code.

In [4]:
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app

Short explaination of the objects:
    
     * gateway - the py4j gateway object representing the connection to the JVM
     * jvm - the object from which you can construct java objects directly from Python
     * app - the entry point, the singleton instance created once at startup.

In [5]:
gateway, jvm, app = connect_jvm(6006)

In [6]:
def get_java_file(path):
    # Here we do the py4j equivalent for new java.io.File(path)
    return jvm.java.io.File(os.path.abspath(path))

## Display available commands

Notebook commands for showing what exists are autogenerated from your classes

In [7]:
app?

Type:           JavaObject
String form:    se.lth.nlp.mentions.App@71a5279d
File:           ~/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py
Docstring:     
Help on class App in package se.lth.nlp.mentions:

App {
|  
|  Methods defined here:
|  
|  buildIndex(File, File) : void
|  
|  getAnalyzer() : Analyzer
|  
|  loadIndex(File) : MentionIndex
|  
|  main(String[]) : void
|  
|  search(MentionIndex, byte[]) : byte[]
|  
|  ------------------------------------------------------------
|  Fields defined here:
|  
|  ------------------------------------------------------------
|  Internal classes defined here:
|  
}
Init docstring:
:param target_id: the identifier of the object on the JVM side. Given
 by the JVM.

:param gateway_client: the gateway client used to communicate with
 the JVM.


## Run commands

Let's build an index using a java method

In [8]:
app.buildIndex(get_java_file("diseases.txt"), get_java_file("disease_index.fst"))

Then load it from disk

In [9]:
indx = app.loadIndex(get_java_file("disease_index.fst"))

## Run mention search from Python

Interacting with java objects is almost the same as with python. The options "auto_convert" and "auto_field" in connect_jvm converts common types such as list, dict to and from equivalent java types.

Except the call overhead to be somewhat high, try to keep number of calls to a minimum, prefer sending a full list to java and do the processing there than calling a method for each entry in the list, unless the list is small or not the computionally dominant part.

In [10]:
hits = indx.search("Acquired Immunodeficiency Syndrome is Alkhurma hemorrhagic fever AHF ALS Amyotrophic Lateral Sclerosis Alzheimer's Disease American Trypanosomiasis Chagas Disease")

hits, indx are all java objects.
hit in hits is a MentionTerm and has fields, we can iterate through java.util.List as if they were python lists.

In [11]:
for hit in hits:
    print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

0 34 Acquired Immunodeficiency Syndrome
38 64 Alkhurma hemorrhagic fever
65 68 AHF
69 72 ALS
73 102 Amyotrophic Lateral Sclerosis
103 122 Alzheimer 's Disease
123 147 American Trypanosomiasis
148 162 Chagas Disease


## Use docria instead

In [12]:
from docria.algorithm import group_by_span, dominant_right_span, dominant_right

Docria provides utilities for storage of processed text. Can save a complicated graph of interconnected nodes, and the stored version can be deserialized in Java and Python at resonable speeds.

There are functions such as group_by_span which can find annotation overlap or coverings, i.e. given a mention, find all terms which intersect it.
dominant_right is a method to remove overlaps by picking the longest to the right in case of conflict.

These functions also exist in the java version where they are usually significantly faster due to the JIT

We create a text with name "main", by convention this is the primary text. You can add as many different texts as you like, texts are special because we can create spans referring to them (range of text).

In [13]:
doc = Document()
doc.add_text("main", "Acquired Immunodeficiency Syndrome is Alkhurma hemorrhagic fever AHF ALS Amyotrophic Lateral Sclerosis Alzheimer's Disease American Trypanosomiasis Chagas Disease")

doc

Document(0 layers, 1 texts)

Zero layers, 1 text.
Now we create a simple way with 3 fields:

 * text - span field into text "main"
 * ident - 64 bit signed integer for a number
 * typename - normal string
 
There are many types, some listed below, for a full reference autocomplete "T" with "T.&lt;TAB&gt;":

 * bool
 * int32
 * int64
 * string
 * binary
 * span
 * noderef - this is a node reference in a layer to another or the same layer.
 * noderef_array - a list of references per node
 
Finally, we will return a node, and in a notebook you will get rich printout of its content

In [14]:
example_layer = doc.add_layer("example", text=T.span("main"), ident=T.int64, typename=T.string)
node = example_layer.add(text=doc.text["main"][0:46], ident=42, typename="DISEASE")
node

Node<example#0>

You can also print out the entire layer

In [15]:
example_layer

Layer(example, N=1)

Get the first node

In [16]:
example_layer[0]

Node<example#0>

In [17]:
doc.printschema()

[example]
 * text     : span[context=main]
 * ident    : i64
 * typename : str



Now we will attempt to use docria to make a full search, it will add 2 layers.

In [18]:
binary_doc = MsgpackCodec.encode(doc) # Encode the document into binary representation
search_binary_doc = app.search(indx, binary_doc) # Process it in java
doc = MsgpackCodec.decode(search_binary_doc) # Decode it back into python
doc.printschema()

[example]
 * ident    : i64
 * text     : span[context=main]
 * typename : str

[matches]
 * id    : i64
 * terms : noderef_array[layer=terms]
 * text  : span[context=main]

[terms]
 * term : str
 * text : span[context=main]
 * type : str



The old data is retained and untouched

In [19]:
example_layer = doc["example"]
example_layer

Layer(example, N=1)

New data has been added

In [20]:
doc["matches"]

Layer(matches, N=8)

In [21]:
doc["terms"]

Layer(terms, N=19)

There is a pythonic way of accessing it

In [22]:
for term in doc["terms"]:
    print(str(term["text"]), "-", "%d:%d" % (term["text"].start, term["text"].stop))

Acquired - 0:8
Immunodeficiency - 9:25
Syndrome - 26:34
is - 35:37
Alkhurma - 38:46
hemorrhagic - 47:58
fever - 59:64
AHF - 65:68
ALS - 69:72
Amyotrophic - 73:84
Lateral - 85:92
Sclerosis - 93:102
Alzheimer - 103:112
's - 112:114
Disease - 115:122
American - 123:131
Trypanosomiasis - 132:147
Chagas - 148:154
Disease - 155:162


We can create docria collections, for parallelized heavy processing as below. This one is compatible with the hadoop readers used in Spark or native Java.

In [23]:
with DocumentIO.write("manydocs.docria") as writer:
    writer.write(doc)

We can also read it again.

In [24]:
for doc in DocumentIO.read("manydocs.docria"):
    print(doc)

== Document ==
                                                                                     Texts                                                                                      

  main    "Acquired Immunodeficiency Syndrome is Alkhurma hemorrhagic fever AHF ALS Amyotrophic Lateral Sclerosis Alzheimer's Disease American Trypanosomiasis Chagas Disease"  

      Layers       

  example    N=1   
  matches    N=8   
  terms      N=19  

